In [1]:
# Importamos librerias
import findspark
findspark.init()
import pyspark
from pyspark.sql import window
from pyspark.sql.functions import lit, trim, col, length, round

# Creamos sesion
from pyspark.sql import SparkSession
spark = (SparkSession
        .builder
        .appName("Padron")
        .getOrCreate())

In [2]:
# Cargamos el csv con la estructura correcta, eliminando "", sustituyendo "" como nulos por 0 y quitando espacios innecesarios
file = 'padron.csv'

padron = (spark.read.format('csv')
         .option('header', 'true')
         .option('inferSchema', 'true')
         .option('delimiter', ';')
         .option('quotes', '\"')
         .option('emptyValue', 0)
         .load(file))
# Para eliminar los espacios en blanco innecesarios:
padron_df = padron.select([(trim(i[0])).alias(i[0]) if i[1] == "string" else i[0] for i in padron.select("*").dtypes])


padron_df.show(10)

+------------+-------------+---------------+-----------+----------+----------------+-----------+------------+----------------+----------------+------------------+------------------+
|COD_DISTRITO|DESC_DISTRITO|COD_DIST_BARRIO|DESC_BARRIO|COD_BARRIO|COD_DIST_SECCION|COD_SECCION|COD_EDAD_INT|EspanolesHombres|EspanolesMujeres|ExtranjerosHombres|ExtranjerosMujeres|
+------------+-------------+---------------+-----------+----------+----------------+-----------+------------+----------------+----------------+------------------+------------------+
|           1|       CENTRO|            101|    PALACIO|         1|            1001|          1|           0|               2|               3|                 1|                 0|
|           1|       CENTRO|            101|    PALACIO|         1|            1001|          1|           1|               7|               0|                 1|                 0|
|           1|       CENTRO|            101|    PALACIO|         1|            1001|      

In [3]:
# Enumerar los distintos barrios
neighborhood = (padron_df.select('DESC_BARRIO')
               .distinct())
neighborhood.show()

+--------------------+
|         DESC_BARRIO|
+--------------------+
|        VALDEFUENTES|
|            ABRANTES|
|       LOS JERONIMOS|
|          NI�O JESUS|
|            VALVERDE|
|              CORTES|
|   PALOMERAS SURESTE|
|CIUDAD UNIVERSITARIA|
|      CUATRO VIENTOS|
|           TRAFALGAR|
|              HELLIN|
|    ALAMEDA DE OSUNA|
|          PRADOLONGO|
|            MOSCARDO|
|          VALDEZARZA|
|           RECOLETOS|
|             HORCAJO|
|        VISTA ALEGRE|
|             EL VISO|
|    PUERTA DEL ANGEL|
+--------------------+
only showing top 20 rows



In [4]:
# Contar a traves de las views, el número de barrios
padron_df.createOrReplaceTempView('padron')

spark.sql('select count(distinct DESC_BARRIO) AS Total_Barrios FROM padron').show()

+-------------+
|Total_Barrios|
+-------------+
|          131|
+-------------+



In [5]:
# Crea una nueva columna que muestre la longitud de los campos de la columna DESC_DISTRITO y que se llame "longitud"
padron_lon = (padron_df
                 .withColumn("Longitud", length(col("DESC_DISTRITO")))
                 .select("Desc_Distrito", "Longitud")
                 .distinct()
                 .show())

+-------------------+--------+
|      Desc_Distrito|Longitud|
+-------------------+--------+
|         ARGANZUELA|      10|
|FUENCARRAL-EL PARDO|      19|
|              USERA|       5|
|          SALAMANCA|       9|
| PUENTE DE VALLECAS|      18|
|  VILLA DE VALLECAS|      17|
|           CHAMBERI|       8|
|          VICALVARO|       9|
|             RETIRO|       6|
|             CENTRO|       6|
|SAN BLAS-CANILLEJAS|      19|
|          CHAMARTIN|       9|
|             LATINA|       6|
|          MORATALAZ|       9|
|            BARAJAS|       7|
|             TETUAN|       6|
|      CIUDAD LINEAL|      13|
|          HORTALEZA|       9|
|         VILLAVERDE|      10|
|        CARABANCHEL|      11|
+-------------------+--------+
only showing top 20 rows



In [6]:
# Crear una columna que muestre el valor 5 para cada uno de los registro de la tabla
padron5 = (padron_df
          .withColumn('Cinco', lit('5'))
          .select('DESC_DISTRITO', 'Cinco')
          .distinct())
padron5.show()

+-------------------+-----+
|      DESC_DISTRITO|Cinco|
+-------------------+-----+
|          VICALVARO|    5|
|             TETUAN|    5|
|           CHAMBERI|    5|
|             RETIRO|    5|
|  VILLA DE VALLECAS|    5|
|         VILLAVERDE|    5|
|        CARABANCHEL|    5|
|              USERA|    5|
|            BARAJAS|    5|
|          CHAMARTIN|    5|
|    MONCLOA-ARAVACA|    5|
|      CIUDAD LINEAL|    5|
|          MORATALAZ|    5|
|         ARGANZUELA|    5|
|          SALAMANCA|    5|
|          HORTALEZA|    5|
|SAN BLAS-CANILLEJAS|    5|
|             CENTRO|    5|
|FUENCARRAL-EL PARDO|    5|
| PUENTE DE VALLECAS|    5|
+-------------------+-----+
only showing top 20 rows



In [7]:
#Borra esta columna
drop_cinco = padron5.drop('Cinco')
drop_cinco.show()

+-------------------+
|      DESC_DISTRITO|
+-------------------+
|          VICALVARO|
|             TETUAN|
|           CHAMBERI|
|             RETIRO|
|  VILLA DE VALLECAS|
|         VILLAVERDE|
|        CARABANCHEL|
|              USERA|
|            BARAJAS|
|          CHAMARTIN|
|    MONCLOA-ARAVACA|
|      CIUDAD LINEAL|
|          MORATALAZ|
|         ARGANZUELA|
|          SALAMANCA|
|          HORTALEZA|
|SAN BLAS-CANILLEJAS|
|             CENTRO|
|FUENCARRAL-EL PARDO|
| PUENTE DE VALLECAS|
+-------------------+
only showing top 20 rows



In [8]:
# Particiona el dataframe por las variables DESC_DISTRITO y DESC_BARRIO
partition = (padron_df
            .repartition(col('DESC_BARRIO'), col('DESC_DISTRITO')))

In [9]:
#Almacénalo en caché. Consulta en el puerto 4040 (UI de Spark) de tu usuario local el estado
# de los rdds almacenados.
partition.cache()
partition.count()
partition.count()

238196

In [10]:
# Lanza una consulta contra el DF resultante en la que muestre el número total de 
#"espanoleshombres", "espanolesmujeres", extranjeroshombres" y "extranjerosmujeres" 
#para cada barrio de cada distrito. Las columnas distrito y barrio deben ser las primeras en 
#aparecer en el show. Los resultados deben estar ordenados en orden de más a menos 
#según la columna "extranjerosmujeres" y desempatarán por la columna 
#"extranjeroshombres".
(partition.select('DESC_DISTRITO', 'DESC_BARRIO', 'EspanolesHombres', 'EspanolesMujeres', 'ExtranjerosHombres', 'ExtranjerosMujeres')
         .groupBy('DESC_DISTRITO', 'DESC_BARRIO')
         .sum('EspanolesHombres', 'EspanolesMujeres', 'ExtranjerosHombres', 'ExtranjerosMujeres')
         .withColumnRenamed('sum(EspanolesHombres)', 'Total Españoles')
         .withColumnRenamed('sum(EspanolesMujeres)', 'Total Españolas')
         .withColumnRenamed('sum(ExtranjerosHombres)', 'Total Extranjeros H')
         .withColumnRenamed('sum(ExtranjerosMujeres)', 'Total Extranjeras F')
         .show())
        

+-------------------+-----------------+---------------+---------------+-------------------+-------------------+
|      DESC_DISTRITO|      DESC_BARRIO|Total Españoles|Total Españolas|Total Extranjeros H|Total Extranjeras F|
+-------------------+-----------------+---------------+---------------+-------------------+-------------------+
|         ARGANZUELA|          ACACIAS|          15399|          18073|               1355|               1507|
|FUENCARRAL-EL PARDO|         VALVERDE|          26922|          29105|               3675|               4441|
|FUENCARRAL-EL PARDO|   FUENTELARREINA|           1445|           1679|                 72|                171|
|          MORATALAZ|          PAVONES|           3584|           4321|                419|                478|
|FUENCARRAL-EL PARDO|        EL GOLOSO|           8947|           9076|                551|                630|
|          HORTALEZA|         CANILLAS|          16770|          19155|               1764|             

In [11]:
# Eliminar el registro caché
partition.unpersist()

DataFrame[COD_DISTRITO: int, DESC_DISTRITO: string, COD_DIST_BARRIO: int, DESC_BARRIO: string, COD_BARRIO: int, COD_DIST_SECCION: int, COD_SECCION: int, COD_EDAD_INT: int, EspanolesHombres: int, EspanolesMujeres: int, ExtranjerosHombres: int, ExtranjerosMujeres: int]

In [12]:
#Crea un nuevo DataFrame a partir del original que muestre únicamente una columna con 
#DESC_BARRIO, otra con DESC_DISTRITO y otra con el número total de "espanoleshombres" 
#residentes en cada distrito de cada barrio. Únelo (con un join) con el DataFrame original a 
#través de las columnas en común.

espanoles = (padron_df.select('DESC_BARRIO', 'DESC_DISTRITO', 'EspanolesHombres')
                     .groupBy('DESC_BARRIO', 'DESC_DISTRITO')
                     .sum('EspanolesHombres')
                     .withColumnRenamed('sum(EspanolesHombres)', 'Españoles')
                     .withColumnRenamed('DESC_BARRIO', 'BARRIO')
                     .withColumnRenamed('DESC_DISTRITO', 'DISTRITO'))
        
(espanoles.join(padron_df,
               (padron_df.DESC_BARRIO == espanoles.BARRIO) & 
               (padron_df.DESC_DISTRITO == espanoles.DISTRITO))
.select('DISTRITO', 'BARRIO', 'Españoles')
.distinct()
.show())


+-------------------+-----------------+---------+
|           DISTRITO|           BARRIO|Españoles|
+-------------------+-----------------+---------+
|         ARGANZUELA|          ACACIAS|    15399|
|FUENCARRAL-EL PARDO|         VALVERDE|    26922|
|FUENCARRAL-EL PARDO|   FUENTELARREINA|     1445|
|          MORATALAZ|          PAVONES|     3584|
|FUENCARRAL-EL PARDO|        EL GOLOSO|     8947|
|          HORTALEZA|         CANILLAS|    16770|
|      CIUDAD LINEAL|SAN JUAN BAUTISTA|     5182|
|          CHAMARTIN|      PROSPERIDAD|    14490|
|      CIUDAD LINEAL|      COSTILLARES|     9778|
|         ARGANZUELA|          LEGAZPI|     8897|
|             LATINA|       CAMPAMENTO|     7102|
|           CHAMBERI|       GAZTAMBIDE|     8921|
|             LATINA| PUERTA DEL ANGEL|    15120|
|        CARABANCHEL|       SAN ISIDRO|    14537|
|          SALAMANCA|             GOYA|    10913|
|             CENTRO|      UNIVERSIDAD|    12410|
| PUENTE DE VALLECAS|         NUMANCIA|    17151|


In [13]:
# Repite la función anterior utilizando funciones de ventana. (over(Window.partitionBy.....)).
